In [560]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import datasets, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import learning_curve, validation_curve
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

pd.options.display.max_rows=260 #60
pd.options.display.max_columns=20 #20

In [561]:
# load data 

trainDF = pd.read_csv("~/Documents/kaggle/bnp/data/train.csv")
testDF = pd.read_csv("~/Documents/kaggle/bnp/data/test.csv")

In [562]:
# eliminate na value more than 120 columns
trainDF = trainDF.dropna(thresh=120)
testDF = testDF.dropna(thresh=120)

In [563]:
testDF.head()

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,0,1.375465e+00,11.361141,C,4.200778,6.577000,2.081784,1.784386,0.011094,9.523810,...,7.619048,1.815241,1.112270e-07,AF,1.292368,3.903345,1.485925,0,2.333334,1.428572e+00
2,2,-4.903407e-07,8.201529,C,4.544371,6.550100,1.558442,2.467532,0.007164,7.142858,...,5.714286,1.970928,1.412265e-02,AV,1.128724,5.844156,1.475892,0,1.263157,-6.380022e-07
3,7,2.661870e+00,3.041241,C,1.657216,9.773080,2.078337,1.430855,1.252157,7.959596,...,4.404040,8.163614,1.100329e+00,B,1.988688,1.558753,2.448814,0,5.385474,1.493777e+00
4,10,1.252822e+00,11.283352,C,4.638388,8.520510,2.302484,3.510159,0.074263,7.612904,...,6.580644,1.325654,2.584588e-01,A,1.863796,2.666478,2.374275,0,0.681672,2.264151e+00
5,11,1.733601e+00,7.525109,C,3.263905,5.608608,2.356090,2.369477,0.065481,9.935483,...,5.032257,2.551374,-9.391195e-07,BM,1.541607,3.463855,1.196959,0,2.598869,1.217392e+00


In [564]:
trainDF.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
6,12,0,0.899806,7.312995,C,3.494148,9.946200,1.926070,1.770427,0.066251,...,3.476299,1.992594,0.083758,BJ,3.276100,1.623298,2.266575,0,2.263736,0.970873
8,22,0,2.078651,8.462619,NaN,3.739030,5.265636,1.573033,2.303371,0.015869,...,8.148148,1.875560,0.018659,S,1.159637,5.582865,1.105283,0,1.170731,3.333334


In [565]:
# defining join, drop, imputation functions

def join(df, col):
    df = df.join(pd.get_dummies(df[col], prefix=col))
    df = df.drop(col, axis=1)
    return df

def drop(df, col):
    df = df.drop(col, axis=1)
    return df

def fillna(df, col):
    df[col] = df[col].fillna(df[col].median(axis=0))
    return df

In [569]:
# v3, 24 30 31 47 52 66 71 74 75 79 91 107 110
# drop 22 38 56 62 113 125 129

list_join = ['v3', 'v24', 'v30', 'v31', 'v47', 'v52', 'v66', 'v71', 'v74', 'v75', 'v79', 'v91', 'v107', 'v110']
list_drop = ['ID', 'v22', 'v38', 'v56', 'v62', 'v113', 'v125', 'v112', 'v129']

trainDF = join(trainDF, list_join)
trainDF = drop(trainDF, list_drop)

testDF = join(testDF, list_join)
testDF = drop(testDF, list_drop)

# get columns index of null value
indexTrain = trainDF.isnull().any()
indexTest = testDF.isnull().any()

# get columns name index
columnsTrain = trainDF.columns
columnsTest = testDF.columns

# create null list
list_index_null_train = []
list_index_null_test = []
list_columns_null_train = []
list_columns_null_test = []

# append to null list index if nan
for i in range(len(indexTrain)):
    if indexTrain[i]: list_index_null_train.append(i)

for i in range(len(indexTest)):
    if indexTest[i]: list_index_null_test.append(i)

list_index_null_test = list(set(map(lambda x: x-2, list_index_null_train) + list_index_null_test))

# append columns name to list
list_columns_null_train = map(lambda x: (columnsTrain[x]), list_index_null_train)
list_columns_null_test = map(lambda x: (columnsTest[x]), list_index_null_test)

# imputation of null value
trainDF = fillna(trainDF, list_columns_null_train)
testDF = fillna(testDF, list_columns_null_test)

In [570]:
# validate null value

print testDF.isnull().values.any()
print trainDF.isnull().values.any()

# print trainDF.columns.values

print list_index_null_test
print list_index_null_train

False
False
[2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 27, 28, 29, 30, 31, 32, 33, 41, 42, 43, 53, 56, 57, 58, 62, 63, 64, 65, 66, 68, 69, 70, 71, 73, 75, 78, 80, 81, 84, 85, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 105, 106, 107, 108]
[4, 8, 9, 11, 13, 15, 19, 20, 21, 29, 31, 34, 43, 44, 55, 58, 59, 64, 65, 66, 67, 70, 72, 75, 82, 86, 89, 91, 92, 95, 97, 99, 101, 102, 103, 107, 109]


In [571]:
# normalize using standard scaler

y_train = trainDF.iloc[:, 0].values
x_train = trainDF.ix[:, 'v1':]
x_test = testDF.ix[:, 'v1':]

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


In [572]:
# feature correlation / importance
forest = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
forest.fit(x_train, y_train)
importances = forest.feature_importances_
print(importances)
indices = np.argsort(importances[::-1])
print indices
# features sort by importances

[  8.19689233e-03   8.03172910e-03   7.78738258e-03   7.59891748e-03
   8.99966919e-03   7.65400896e-03   6.84015538e-03   7.97227864e-03
   1.41661813e-02   7.73324368e-03   1.59212485e-02   7.50814586e-03
   1.27292942e-02   7.79740908e-03   8.26815142e-03   6.95598569e-03
   8.25654648e-03   7.70525151e-03   7.92538197e-03   1.12420364e-02
   3.03191710e-03   7.15176624e-03   7.49362657e-03   7.68956708e-03
   8.51432084e-03   6.72404472e-03   7.46271850e-03   7.23393060e-03
   1.23897107e-02   7.86210568e-03   7.81583478e-03   8.19942982e-03
   8.01000525e-03   1.20870737e-02   6.86917523e-03   7.75633845e-03
   7.31998257e-03   7.97838392e-03   8.12088928e-03   7.03000058e-03
   6.84736141e-03   7.26843725e-03   5.02767281e-02   7.68375956e-03
   7.77479437e-03   7.56925903e-03   7.72734532e-03   8.50237151e-03
   7.54654532e-03   7.58422031e-03   7.54872423e-03   7.45697235e-03
   7.24436668e-03   6.77578391e-03   7.19436762e-03   6.90544010e-03
   8.61510257e-03   8.61017000e-03